In [32]:
# Data preprocessing, math and plotting
import os
import numpy as np 
import pandas as pd 
from scipy import stats 
import matplotlib.pyplot as plt
import seaborn as sns 
# ML 
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix

In [33]:
customers_df = pd.read_csv(r"data\olist_customers_dataset.csv")
geolocation_df = pd.read_csv(r'data/olist_geolocation_dataset.csv') # drop
orders_df = pd.read_csv(r'data/olist_orders_dataset.csv')
order_items_df = pd.read_csv(r'data/olist_order_items_dataset.csv')
order_payments_df = pd.read_csv(r'data/olist_order_payments_dataset.csv')
reviews_df = pd.read_csv(r'data/olist_order_reviews_dataset.csv') 
products_df = pd.read_csv(r'data/olist_products_dataset.csv')
sellers_df = pd.read_csv(r'data/olist_sellers_dataset.csv') 
category_names_translated_df = pd.read_csv(r'data/product_category_name_translation.csv') 
# Get product category translation
products_translated_df = category_names_translated_df.merge(products_df, on='product_category_name', how='left')
products_translated_df.drop(products_translated_df.columns[[0]], axis=1, inplace=True)

ValueError: Only callable can be used as callback

In [ ]:
merged_df = orders_df.merge(order_items_df, on='order_id', how='left')
merged_df = merged_df.merge(order_payments_df, on='order_id', how='outer', validate='m:m')
merged_df = merged_df.merge(reviews_df, on='order_id', how='outer')
merged_df = merged_df.merge(customers_df, on='customer_id', how='outer')
merged_df = merged_df.merge(sellers_df, on='seller_id', how='outer')
merged_df = merged_df.merge(products_translated_df, on='product_id', how='outer')

### Create Model-based Collaborative filtering system for Product Recommendation

##### Recommend items to customers based on the purchase history and similarity of ratings provided by other customers. Items bought by the same customers, with the highest ratings, will be recommended to other customers with similar purchase history, weighted by their item rankings. Using a collaborative filtering technique helps predict products that a customer might buy, based on the patterns seen by customer specific preferences.

##### Utility Matrix & Singular Value Decomposition (SVD)
###### A utility matrix consits of all possible customer-item preferences, using item similarity weighted by the customer's ratings of said items. The data associated with each customer represents how much the customer appreciates the item, in respect to others who purchase the same item.
###### The SVD transformer performs linear dimensionality reduction by means of truncated singular value decomposition (SVD). Unlike PCA, principle component analysis, this estimator does not center the data before computing the singular value decomposition. This means that it can efficiently work with sparse matices.

In [31]:
# subset of dataset 'df_train'
df_train_subset = merged_df.head(10000)

df_train_util_matrix = df_train_subset.pivot_table(values='review_score', index='customer_id', columns='product_id', fill_value=0)
df_train_util_matrix.head()
            
# transpose the matrix
X = df_train_util_matrix.T
X.head()

# unique products in subset of data
X1 = X

# SVD transformation
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
print(decomposed_matrix.shape)

# Correlation Matrix
correlation_matrix = np.corrcoef(decomposed_matrix)

# Isolating Product ID # 009c09f439988bc06a93d6b8186dce73 from the Correlation Matrix (random selection based on index value[1])
product = X.index[1]

[autoreload of numpy.core.multiarray failed: Traceback (most recent call last):
  File "C:\Users\trevor.sauerbrey\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\trevor.sauerbrey\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 471, in superreload
    module = reload(module)
  File "C:\Program Files\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "C:\Users\trevor.sauerbrey\AppData\Roaming\Python\Python310\site-packages\numpy\core\multiarray.py", line 16, in <module>
    from ._multiarray_umath import (
ImportError: cannot import name 'fastCopyAndTranspose' from 'numpy.core._mu

TypeError: _vhstack_dispatcher() missing 2 required keyword-only arguments: 'dtype' and 'casting'

In [28]:
# Index number and product ID purchased by customer 
i = '009c09f439988bc06a93d6b8186dce73'

product_names = list(X.index)
product_id = product_names.index(i)

# Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product
correlation_product_ID = correlation_matrix[product_id]
correlation_product_ID.shape

# Recommending the most highly correlated products in sequence with scores of over 0.90
Recommend = list(X.index[correlation_product_ID > 0.90])

# Removes the item already bought by the customer 
Recommend.remove(i) 

#### Below are the top products to be displayed by the recommendation system to the above customer based on the purchase history of other customers in the website (Correlation score over 90%)
#### (only if we knew what each product ID represented....)

In [26]:
Recommend

['060890b1af8cc031e9dde49cf544f884',
 '0f5d0170c03b333aa1a44fa11e040e75',
 '1468ae815f666819cd3fa15be0cc035a',
 '1cf4209f5358de32c5cae2c2dbb84aed',
 '1d3d7e2bc2cb75e4e2c798ec9aae0279',
 '2362291929f77eccf5f5cb26029dcf90',
 '2827ebfb043a328a13a06ca9165cad0a',
 '3b0fbb3ab5b1512278d2e1c695b07e85',
 '3d250bb3d2b89c9bea15327aed52d8ca',
 '4ae634441e444ca4bc85903cafe98d73',
 '4c1bbc12438daec98a77243c2bf7a3ba',
 '59c5e6e108b88212fb98525c9000ecfb',
 '5fb8e63ae576fbfbd243c50c5ce74050',
 '716854aa9d670cabfec04c7eeddd85b8',
 '802b191477641b98e903230bcfaa2a47',
 '873eb5f3b8cc503730e472a14cd26616',
 '8e87b57f5bc7661130ee7803017f3f32',
 '9aa3269c14e72d138d7e346897444d0f',
 'de5596e009313841e57616878a6fbce0',
 'ea17b2137544a3b0a3eaf107887472d9',
 'f130542b9dab05d4540ba00f35f0880f',
 'fca5d31cf768e5bf8a75483fa7f4674c']